In [1]:
import os
import folium

print(folium.__version__)

0.3.0+206.g2acfc64.dirty


In [2]:
from folium.plugins import MarkerCluster


m = folium.Map(location=[44, -73], zoom_start=5)

marker_cluster = MarkerCluster().add_to(m)


folium.Marker(
    location=[40.67, -73.94],
    popup='Add popup text here.',
    icon=folium.Icon(color='green', icon='ok-sign'),
).add_to(marker_cluster)

folium.Marker(
    location=[44.67, -73.94],
    popup='Add popup text here.',
    icon=folium.Icon(color='red', icon='remove-sign'),
).add_to(marker_cluster)

folium.Marker(
    location=[44.67, -71.94],
    popup='Add popup text here.',
    icon=None,
).add_to(marker_cluster)

m.save(os.path.join('results', '1000_MarkerCluster0.html'))

m

In [3]:
import numpy as np

size = 100
lons = np.random.randint(-180, 180, size=size)
lats = np.random.randint(-90, 90, size=size)

locations = list(zip(lats, lons))
popups = ['lon:{}<br>lat:{}'.format(lon, lat) for (lat, lon) in locations]

Adding all icons in a single call

In [4]:
icon_create_function = """\
function(cluster) {
    return L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className: 'marker-cluster marker-cluster-large',
    iconSize: new L.Point(20, 20)
    });
}"""

In [5]:
from folium.plugins import MarkerCluster


m = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    tiles='Cartodb Positron',
    zoom_start=1
)

marker_cluster = MarkerCluster(
    locations=locations, popups=popups,
    name='1000 clustered icons',
    overlay=True,
    control=True,
    icon_create_function=icon_create_function
)

marker_cluster.add_to(m)

folium.LayerControl().add_to(m)

m.save(os.path.join('results', '1000_MarkerCluster1.html'))

m

Explicit loop allow for customization in the loop.

In [6]:
%%time

m = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    tiles='Cartodb Positron',
    zoom_start=1
)

marker_cluster = MarkerCluster(
    name='1000 clustered icons',
    overlay=True,
    control=False,
    icon_create_function=None
)

for k in range(size):
    location = lats[k], lons[k]
    marker = folium.Marker(location=location)
    popup = 'lon:{}<br>lat:{}'.format(location[1], location[0])
    folium.Popup(popup).add_to(marker)
    marker_cluster.add_child(marker)

marker_cluster.add_to(m)

folium.LayerControl().add_to(m)

CPU times: user 2.04 s, sys: 3.93 ms, total: 2.04 s
Wall time: 2.04 s


In [7]:
m.save(os.path.join('results', '1000_MarkerCluster2.html'))

m

`FastMarkerCluster` is not as flexible as MarkerCluster but, like the name suggests, it is faster.

In [8]:
from folium.plugins import FastMarkerCluster

In [9]:
%%time


m = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    tiles='Cartodb Positron',
    zoom_start=1
)

FastMarkerCluster(data=list(zip(lats, lons))).add_to(m)

folium.LayerControl().add_to(m)

CPU times: user 82.6 ms, sys: 0 ns, total: 82.6 ms
Wall time: 81.3 ms


In [10]:
m.save(os.path.join('results', '1000_MarkerCluster3.html'))

m

In [11]:
callback = """\
function (row) {
    var icon, marker;
    icon = L.AwesomeMarkers.icon({
        icon: "map-marker", markerColor: "red"});
    marker = L.marker(new L.LatLng(row[0], row[1]));
    marker.setIcon(icon);
    return marker;
};
"""
m = folium.Map(
    location=[np.mean(lats), np.mean(lons)],
    tiles='Cartodb Positron',
    zoom_start=1
)

FastMarkerCluster(
    data=list(zip(lats, lons)),
    callback=callback
).add_to(m)


m.save(os.path.join('results', '1000_MarkerCluster4.html'))

m